# Personalização de Engajamento de Clientes no Santander usando IA Generativa

O intuito desse notebook é implementar uma ETL que personalizada mensagens de marketing para clientes do Santander usando IA Generativa

## Contexto

Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.



1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.

## Instação e importação das bibliotecas

In [1]:
!pip install requests pandas openai tqdm bardapi --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.

In [2]:
import requests
import pandas as pd
import time
import os
import time
import json
from tqdm import tqdm
from bardapi import Bard

## Configuração do ambiente

**Começamos** dando um valor para
`'

BARD_API_KEY'`, que é a key para uso do API do Google Bard. É possível encontrar mais detalhes no link do [repositório GitHub do bard-api](https://github.com/dsdanielpark/Bard-API).

In [3]:
# Replace XXXX with the values you get from __Secure-1PSID
os.environ['_BARD_API_KEY']="WCwHxs2yondbQaEc/AGBT507znC7dUDkkE"

Definindo o máximo valor de tentativas para buscar os IDs na API e o delay inicial para começar.

In [4]:
MAX_ATTEMPTS = 3
INITIAL_DELAY = 2

## Carregamento de IDs


O arquivo de origem é um csv composto por:

  ```
  UserID
  1
  2
  3
  4
  5
  ```

In [6]:
# Carregamento de arquivos (para o csv)
from google.colab import files

uploaded = files.upload()

Saving Santander_Dev_Week2023.csv to Santander_Dev_Week2023.csv


In [8]:
file_name = list(uploaded.keys())[0]
print(file_name)

Santander_Dev_Week2023.csv


In [10]:
#Função para carregar os valores na variavel
def load_user_ids(filename):
    return pd.read_csv(filename)['UserID']

user_ids = load_user_ids(f"{file_name}")
print(f"User IDs loaded: {user_ids.tolist()}\n")

User IDs loaded: [1, 2, 3, 4, 5]



# Interação API

# Exploração dos dados

Aqui, a função "get_user-data" tem como intuito fazer a ligação entre o sistema e a API

In [11]:
def get_user_data(user_id):
    url = f"https://sdw-2023-prd.up.railway.app/users/{user_id}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()
    except requests.RequestException as e:
        return None

test_response = get_user_data(1)
print(json.dumps(test_response, indent = 4))

{
    "id": 1,
    "name": "Devweekerson",
    "account": {
        "id": 1,
        "number": "01.097954-4",
        "agency": "2030",
        "balance": 624.12,
        "limit": 1000.0
    },
    "card": {
        "id": 1,
        "number": "xxxx xxxx xxxx 1111",
        "limit": 2000.0
    },
    "features": [
        {
            "id": 2,
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pay.svg",
            "description": "Pagar"
        },
        {
            "id": 3,
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/transfer.svg",
            "description": "Transferir"
        },
        {
            "id": 4,
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/account.svg",
            "description": "Conta Corrente"
        },
        {
            "id": 5,
            "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/

# Transformação dos dados

In [12]:
def generate_custom_message(user_data):
    variables = {
        "name": user_data.get('name', 'Cliente'),
        "balance": user_data.get('account', {}).get('balance', 0),
        "card_limit": user_data.get('card', {}).get('limit', 0)
    }

    if variables['balance'] < 1000:
        balance_msg = "Começar a investir com recursos financeiros limitados pode ser um desafio, porém, não é uma tarefa impossível."
    else:
        balance_msg = "Com a sua sólida situação financeira, as possibilidades de investimento se expandem consideravelmente."

    if variables['card_limit'] < 1000:
        card_msg = "Devido ao seu limite de cartão mais modesto, é aconselhável considerar opções de investimento com menor grau de risco."
    else:
        card_msg = "Graças ao seu limite de cartão mais alto, você tem a liberdade de explorar uma ampla gama de oportunidades de investimento."

    return f"Olá {variables['name']}," + "\n" + f"{balance_msg}" + "\n" + f"{card_msg}" + "Investir é a chave para fazer o seu dinheiro crescer. Não deixe seus recursos ociosos. Está pronto para começar a investir?"

In [13]:
# Função para geração da mensagem
def generate_bard_message(user_data):
    bard = Bard()
    name = user_data.get('name', 'Cliente')
    balance = user_data.get('account', {}).get('balance', 0)
    card_limit = user_data.get('card', {}).get('limit', 0)
    prompt = f"Você é um banqueiro expert, que lida com investimentos e aconselhamento. Crie uma mensagem para {name} sobre a importância dos investimentos. Considere que o saldo deles é de {balance} e o limite do cartão de crédito de {card_limit}. (responda somente com a mensagem)"

    return bard.get_answer(prompt)['content']

# Carregamento dos dados


In [15]:
def print_colored_message(title, message = None, custom_message = None, bard_message = None, mType = "warning"):
    # ANSI escape codes for color and background
    base_bg_color = "\033[100m"  # original gray background
    color_map = {
        "warning": "\033[93m" + base_bg_color,  # yellow text, gray background
        "success": "\033[92m" + base_bg_color,  # green text, gray background
        "error": "\033[91m" + base_bg_color,    # red text, gray background
    }
    title_colors = "\033[91m" + base_bg_color  # light red text for titles
    bard_title_colors = "\033[96m" + base_bg_color  # cyan text for bard title
    colors = color_map.get(mType, "\033[0m")

    def print_fixed_width_line(line, line_color=colors):
        print(line_color + f"{line}{' ' * (80 - len(line))}\033[0m")

    print_fixed_width_line("="*80)
    print_fixed_width_line(title.center(80))
    print_fixed_width_line("="*80)

    if message:
        print_fixed_width_line(message)
        print_fixed_width_line("="*80)

        return


    print_fixed_width_line("Resposta baseada em regras:", title_colors)
    print_fixed_width_line("")

    for line in custom_message.split("\n"):
        words = line.split()
        output_line = ""
        for word in words:
            if len(output_line) + len(word) + 1 > 80:
                print_fixed_width_line(output_line.strip())
                output_line = ""
            output_line += (word + " ")
        print_fixed_width_line(output_line.strip())

    print_fixed_width_line("")


    print_fixed_width_line("Mensagem do Bard:", bard_title_colors)
    print_fixed_width_line("")

    for line in bard_message.split("\n"):
        words = line.split()
        output_line = ""
        for word in words:
            if len(output_line) + len(word) + 1 > 80:
                print_fixed_width_line(output_line.strip())
                output_line = ""
            output_line += (word + " ")
        print_fixed_width_line(output_line.strip())

    print_fixed_width_line("")

# Processamento principal

In [17]:
def process_single_user(user_id, retry_delay=INITIAL_DELAY):
    pbar = tqdm(range(MAX_ATTEMPTS), desc=f"Processing User {user_id}", leave=False)

    for attempt in pbar:
        try:
            user_data = get_user_data(user_id)

            if user_data is None:
                raise Exception("Dados do usuário não disponíveis")

            custom_message = generate_custom_message(user_data)
            bard_message = generate_bard_message(user_data)

            print_colored_message(f"Recomendações para o usuário {user_id}", custom_message=custom_message, bard_message=bard_message, message_type="warning")


            pbar.n = MAX_ATTEMPTS
            pbar.last_print_n = MAX_ATTEMPTS
            pbar.update()
            pbar.close()

            return user_data, custom_message, bard_message
        except Exception as e:
            time.sleep(retry_delay)
            retry_delay *= 2

    pbar.close()

    return None, None, None

In [18]:
user_messages_dict = {}

In [21]:
def process_users(user_ids):
    for user_id in user_ids:
        user_data, custom_message, bard_message = process_single_user(user_id)

        user_messages_dict[user_id] = {
            'user_data': user_data,
            'custom_message': custom_message,
            'bard_message': bard_message
        }

        if user_data is None:
            print("\n" + f"Falha ao processar o usuário {user_id}.")
        else:
            print("\n" + f"Usuário {user_id} processado com sucesso.")

    print("Lista de usuários finalizada.")


In [22]:
process_users(user_ids)


Falha ao processar o usuário 1.



Falha ao processar o usuário 2.



Falha ao processar o usuário 3.



Falha ao processar o usuário 4.



Falha ao processar o usuário 5.
Lista de usuários finalizada.


# Atualização dos dados

In [23]:
def update_user(user_id, user_data, bard_message):
    # URL base da API
    sdw2023_api_url = "https://sdw-2023-prd.up.railway.app"

    # Preparar a nova mensagem
    new_news_item = {
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": bard_message
    }

    # Adicionar a nova mensagem à lista de "news" existente
    if 'news' in user_data:
        user_data['news'].append(new_news_item)
    else:
        user_data['news'] = [new_news_item]

    # Fazer a requisição PUT para atualizar o usuário
    response = requests.put(f"{sdw2023_api_url}/users/{user_id}", json=user_data)

    # Retornar o status da operação
    return True if response.status_code == 200 else False

In [24]:
def update_all_users():
    for user_id, user_entry in user_messages_dict.items():
        user_data = user_entry['user_data']
        bard_message = user_entry['bard_message']

        if user_data is not None:  # Skip entries where user_data is None
            success = update_user(user_id, user_data, bard_message)

            # Use a cor verde para sucesso e vermelha para falha
            message_type = "success" if success else "error"

            print_colored_message(
                title=f"Status de Atualização para o Usuário {user_id}",
                message=f"Usuário {user_id} atualizado? {success}",
                mType=message_type
            )

In [25]:
update_all_users()